In [7]:
!pwd

/home/furfa/work/ai-academy2019/notebooks


In [8]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
import os
pd.set_option("max_columns", 10000)

%pylab inline
import seaborn as sns

import warnings
warnings.simplefilter("ignore")

import json
from IPython import display

from tqdm import tqdm

Populating the interactive namespace from numpy and matplotlib


In [9]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [10]:
base_dir = ".."
input_dir = open( os.path.join(base_dir, "datadir.txt"), "r").read()[:-1]
pred_dir = os.path.join(base_dir, "predictions")
os.listdir( input_dir )

['ai-academy-2019-skill-prediction-data-jsonlines.zip',
 'ai-academy-2019-skill-prediction-data-csv-v1.zip',
 'dota2_items.csv',
 'dota2_heroes.csv',
 'dota2_skill_test.csv',
 'processed',
 'dota2_skill_train.csv',
 'dota2_skill_test.jsonlines',
 'dota2_skill_train.jsonlines',
 'dota2_abilities.csv']

In [11]:
with open(os.path.join(input_dir, "dota2_skill_test.jsonlines")) as file:
    for line in file:
        record = json.loads(line)
        # обработка record
        
        break

In [12]:
len(record) - 8

57

In [13]:
display.JSON(record)

<IPython.core.display.JSON object>

In [14]:
def process_line(record):
    ret = dict()
    
    ret["id"] = record["id"]
    # team stats
    for i, it in enumerate(record['radiant_heroes']):
        ret[f"radiant_hero_{i}"] = it
        
    for i, it in enumerate(record['dire_heroes']):
        ret[f"dire_hero_{i}"] = it
    
    ret["radiant_final_gold"] = record['series']['radiant_gold'][-1]
    ret["dire_final_gold"] = record['series']['dire_gold'][-1]
    
    ret["n_items"] = len(record['item_purchase_log'])
    
    return ret
process_line(record)

{'id': 0,
 'radiant_hero_0': 70,
 'radiant_hero_1': 53,
 'radiant_hero_2': 74,
 'radiant_hero_3': 7,
 'radiant_hero_4': 64,
 'dire_hero_0': 68,
 'dire_hero_1': 23,
 'dire_hero_2': 38,
 'dire_hero_3': 100,
 'dire_hero_4': 12,
 'radiant_final_gold': 117912,
 'dire_final_gold': 105122,
 'n_items': 37}

In [15]:
dire = record['series']['dire_gold']
radiant = record['series']['radiant_gold']
time = np.arange(len(dire))

trace_d = go.Scatter(
    x = time,
    y = dire,
    name = "dire"
)
trace_r = go.Scatter(
    x = time,
    y = radiant,
    name = "radiant"
)

layout = go.Layout(
    title='radiant_dire_gold',
)

fig = go.Figure(data = [trace_d, trace_r], layout = layout)
iplot(fig)

In [16]:
data = record['series']['player_gold']
time = np.arange(len(data))

trace = go.Scatter(
    x = time,
    y = data,
    name = "dire"
)

layout = go.Layout(
    title='player_gold',
)

fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

### Добавить времена апов

* Герои комманды и противники
* Синергии
* Скорость набора голды и статистики по голде
* Вероятоность победы по пику
* Ult upgrades
* Вычесть на каком уровне держит голду
* Дисперсии

In [77]:
def process_file(f_name):
    with open(f_name) as file:
        answer = list()
        for line in tqdm(file):
            rec= json.loads(line)
            # обработка record
            answer.append(
                process_line(rec)
            )
    return pd.DataFrame(answer).set_index("id")

In [78]:
# Processing
train = process_file(os.path.join(input_dir, "dota2_skill_train.jsonlines"))
test = process_file(os.path.join(input_dir, "dota2_skill_test.jsonlines"))

99871it [00:47, 2086.93it/s]
43265it [00:20, 2100.31it/s]


In [82]:
try:
    processed_dir = os.path.join(input_dir, "processed")
    os.mkdir(processed_dir)
except:
    pass
finally:
    train.to_csv(
        os.path.join(processed_dir, "train_proc.csv")
    )
    test.to_csv(
        os.path.join(processed_dir, "test_proc.csv")
    )